In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --pre torch torchvision torchtext -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html -U
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip install 'git+https://github.com/facebookresearch/mobile-vision.git'
!git clone https://github.com/facebookresearch/d2go
!pip install "/content/d2go"
!pip install 'git+https://github.com/facebookresearch/mobile-vision.git'
!pip install pyyaml==5.4
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!pip install numpy

In [ ]:
import torch, torchvision
print(torch.version, torch.cuda.is_available())

<module 'torch.version' from '/usr/local/lib/python3.9/dist-packages/torch/version.py'> True


In [ ]:
!cp -av '/content/d2go/configs' '/usr/local/lib/python3.9/dist-packages/d2go/'

In [ ]:
from d2go.model_zoo import model_zoo
model = model_zoo.get('mask_rcnn_fbnetv3a_C4.yaml', trained=True)

model_final.pth: 74.3MB [00:06, 11.0MB/s]                            


In [ ]:
!python --version

Python 3.9.16


In [ ]:
import cv2
from matplotlib import pyplot as plt
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
im = cv2.imread("./input.jpg")
plt.imshow(im)

In [ ]:
from d2go.utils.demo_predictor import DemoPredictor
predictor = DemoPredictor(model)
outputs = predictor(im)
# the output object categories and corresponding bounding boxes
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

tensor([17,  0,  0, 17, 17, 17,  0, 20, 19,  0, 17, 20, 17, 22, 19, 19, 19],
       device='cuda:0')
Boxes(tensor([[123.8110, 241.3701, 479.6925, 480.0000],
        [276.2808, 147.8152, 365.5569, 400.5892],
        [ 51.7697, 279.1486,  78.7433, 341.4369],
        [504.6472, 261.5977, 587.4645, 335.1068],
        [331.7795, 234.2218, 410.3190, 307.9734],
        [408.7300, 285.3493, 462.2014, 353.3682],
        [  2.9709, 277.4141,  75.1721, 476.9530],
        [  3.1935, 277.9557,  75.0489, 476.4275],
        [504.8673, 262.9518, 592.4090, 339.0172],
        [556.2568, 270.3552, 595.2379, 355.6459],
        [  3.2859, 277.9533,  75.0428, 476.7831],
        [504.8312, 262.8539, 587.5267, 330.3096],
        [555.7526, 280.5878, 595.2996, 360.6277],
        [555.3569, 279.8503, 595.3765, 360.5094],
        [595.2652, 266.1639, 628.4905, 333.6963],
        [  2.7089, 277.7745,  75.3923, 476.8129],
        [555.5452, 276.6618, 598.2251, 355.9480]], device='cuda:0'))


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get("coco_2017_train"))
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
DatasetCatalog.clear()

for 500 train images

In [ ]:
'''from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "train .json file", "train image dataset link")
register_coco_instances("my_dataset_val", {}, "validation.json file", "valid_images link")'''

In [ ]:
#visualize training data
from detectron2.utils.visualizer import Visualizer

In [ ]:
my_dataset_val_metadata = MetadataCatalog.get("my_dataset_val")
dataset_dicts = DatasetCatalog.get("my_dataset_val")

import random
dataset_dicts = DatasetCatalog.get("my_dataset_val")
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_val_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    plt.figure()
    plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
#visualize training data
from detectron2.utils.visualizer import Visualizer

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
dataset_dicts = DatasetCatalog.get("my_dataset_train")
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    plt.figure()
    plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
wep_train_metadata = MetadataCatalog.get("my_dataset_train")
wep_train_metadata

In [ ]:
wep_val_metadata = MetadataCatalog.get("my_dataset_val")
wep_val_metadata

In [ ]:
import os
from d2go.runner import GeneralizedRCNNRunner

In [ ]:
def prepare_for_launch():
    runner = GeneralizedRCNNRunner()
    cfg = runner.get_default_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("faster_rcnn_fbnetv3a_C4.yaml"))
    cfg.DATASETS.TRAIN = ("my_dataset_train",)
    cfg.DATASETS.TEST = ("my_dataset_val",)

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("faster_rcnn_fbnetv3a_C4.yaml")  # Let training initialize from model zoo
    cfg.SOLVER.IMS_PER_BATCH = 1
    cfg.SOLVER.BASE_LR = 0.0001


    cfg.SOLVER.WARMUP_ITERS = 20
    cfg.SOLVER.MAX_ITER = 500 #adjust up if val mAP is still rising, adjust down if overfit
    cfg.SOLVER.STEPS = []
    cfg.SOLVER.GAMMA = 0.05

    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

    cfg.TEST.EVAL_PERIOD = 10
    return cfg, runner

cfg, runner = prepare_for_launch()
model = runner.build_model(cfg)
runner.do_train(cfg, model, resume=False)

In [ ]:
metrics = runner.do_test(cfg, model)

In [ ]:
print(metrics)

In [ ]:
import copy
from detectron2.data import build_detection_test_loader
from d2go.export.exporter import convert_and_export_predictor
from d2go.utils.testing.data_loader_helper import create_detection_data_loader_on_toy_dataset

import logging

# disable all the warnings
previous_level = logging.root.manager.disable
logging.disable(logging.INFO)

cfg_name = 'faster_rcnn_fbnetv3a_C4.yaml'
pytorch_model = model_zoo.get(cfg_name, trained=True, device='cpu')
pytorch_model.eval()
cfg = model_zoo.get_config(cfg_name)

with create_detection_data_loader_on_toy_dataset(cfg, 224, 320, is_train=False) as data_loader:
    predictor_path = convert_and_export_predictor(
            cfg,
            copy.deepcopy(pytorch_model),
            "torchscript_int8",
            './',
            data_loader,
        )

# recover the logging level
logging.disable(previous_level)

In [ ]:
from d2go.utils.demo_predictor import DemoPredictor

predictor = DemoPredictor(model)

dataset_dicts = DatasetCatalog.get('my_dataset_val')
for i in random.sample(dataset_dicts, 5):
    im = cv2.imread(i["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("my_dataset_val"), scale=0.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
from typing import List, Dict
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile
import os

predictor_path = "/content/torchscript_int8"

class Wrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        coco_idx_list = [1, 2, 3, 4, 5]
        self.coco_idx = torch.tensor(coco_idx_list)

    def forward(self, inputs: List[torch.Tensor]):
        x = inputs[0].unsqueeze(0) * 255
        scale = 320.0 / min(x.shape[-2], x.shape[-1])
        x = torch.nn.functional.interpolate(x, scale_factor=scale, mode="bilinear", align_corners=True, recompute_scale_factor=True)
        out = self.model(x[0])
        res : Dict[str, torch.Tensor] = {}
        res["boxes"] = out[0] / scale
        res["labels"] = torch.index_select(self.coco_idx, 0, out[1])
        res["masks"] = out[2]
        res["scores"] = out[3]
        return inputs, [res]

orig_model = torch.jit.load(os.path.join(predictor_path, "model.jit"))
wrapped_model = Wrapper(orig_model)
scripted_model = torch.jit.script(wrapped_model)
# scripted_model.save("d2go_mask.pt")
optimized_scripted_module = optimize_for_mobile(scripted_model)

#the name of the converted model to be used for android
optimized_scripted_module._save_for_lite_interpreter("d2go_josh_faster.ptl")